In [77]:
import pandas as pd 
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers 
from keras_tuner.tuners import RandomSearch

In [78]:
file_path = "./Desktop/datasets/Real_Combine.csv"

In [79]:
data = pd.read_csv(file_path)

In [80]:
data.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [81]:
X = data.iloc[:, :-1] ## independent features 
y = data.iloc[:, -1] ## dependent features 


In [82]:
X.head()

,T,TM,Tm,SLP,H,VV,V,VM
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2


In [83]:
y.head()

0    219.720833
1    182.187500
2    154.037500
3    223.208333
4    200.645833
Name: PM 2.5, dtype: float64

In [84]:
X.shape

(1093, 8)

In [102]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Dense(units = 8, activation = "relu"))
    model.add(
        layers.Dense(
            # Define the hyperparameter.
            units=hp.Int("units", min_value=32, max_value=512, step=32),
            activation="relu",
        )
    )
    model.add(layers.Dense(1, activation="linear"))
    model.compile(
        optimizer="adam", loss="mean_squared_error", metrics=[tf.keras.metrics.MeanSquaredError()],
    )
    return model

In [103]:
tuner = RandomSearch(
    build_model,
    objective="mean_squared_error",
    max_trials=5)

In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [105]:
tuner.search(X_train, y_train, epochs = 5)

Trial 5 Complete [00h 00m 00s]
mean_squared_error: 7187.9267578125

Best mean_squared_error So Far: 6862.33740234375
Total elapsed time: 00h 00m 02s
INFO:tensorflow:Oracle triggered exit


In [106]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
units: 384
Score: 6862.33740234375
Trial summary
Hyperparameters:
units: 64
Score: 7037.3828125
Trial summary
Hyperparameters:
units: 160
Score: 7060.6826171875
Trial summary
Hyperparameters:
units: 352
Score: 7087.3798828125
Trial summary
Hyperparameters:
units: 448
Score: 7187.9267578125


In [107]:
models = tuner.get_best_models(num_models = 3)

In [108]:
best = models[0]

In [109]:
best.build(input_shape = (None, 8))

In [110]:
best.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 384)               3456      
                                                                 
 dense_2 (Dense)             (None, 1)                 385       
                                                                 
Total params: 3,913
Trainable params: 3,913
Non-trainable params: 0
_________________________________________________________________
